In [ ]:
import cv2, os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
for file in os.listdir('Images'):
    img = cv2.imread(f'Images/{file}', cv2.IMREAD_UNCHANGED) 
    src = img.copy()
    src[:,:,2] = np.zeros([src.shape[0], src.shape[1]])
    src[:,:,0] = np.zeros([src.shape[0], src.shape[1]])

    gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
    _,edge = cv2.threshold(gray, 50, 1, cv2.THRESH_BINARY)
    contours,_ = cv2.findContours(edge,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    nb_cont = 0
    best = -1
    for i, contour in enumerate(contours):
        if len(contours[i]) > nb_cont:
            nb_cont = len(contours[i])
            best = i

    xmin, xmax, ymin, ymax = 5000, 0, 5000, 0
    for i, contour in enumerate(contours[best]):
        if contour[0][0] > xmax:
            xmax = contour[0][0] 

        if contour[0][0] < xmin:
            xmin = contour[0][0] 

        if contour[0][1] > ymax:
            ymax = contour[0][1] 

        if contour[0][1] < ymin:
            ymin = contour[0][1]


    cropped = img[ymin:ymax, xmin:xmax]
    resized = cv2.resize(cropped, (640, 640), interpolation = cv2.INTER_AREA)

    cv2.imwrite(f'New_Images/{file}', resized)